# Faster-RCNN Model Implemented using PyTorch

***

**Author:** Shane Cooke

**Date:** 30 Sept 2022

**References:**
* https://medium.com/fullstackai/how-to-train-an-object-detector-with-your-own-coco-dataset-in-pytorch-319e7090da5
* https://towardsdatascience.com/train-mask-rcnn-net-for-object-detection-in-60-lines-of-code-9b6bbff292c3
* https://towardsdatascience.com/how-to-use-datasets-and-dataloader-in-pytorch-for-custom-text-data-270eed7f7c00
* https://www.kaggle.com/code/abhishek/train-your-own-mask-rcnn/notebook

***

## Imports & Data Preparation

#### Google Colab

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#cd drive/MyDrive/MastersProject/Code/

In [3]:
#gpu_info = !nvidia-smi
#gpu_info = '\n'.join(gpu_info)
#if gpu_info.find('failed') >= 0:
#  print('Not connected to a GPU')
#else:
#  print(gpu_info)

In [4]:
#!nvcc --version

#### Install Dependencies

In [5]:
#!pip install albumentations==0.4.6
#!pip install pycocotools --quiet

# Clone TorchVision repo and copy helper files
#!git clone https://github.com/pytorch/vision.git
#%cd vision
#!git checkout v0.3.0
#%cd ..
#!cp vision/references/detection/utils.py ./
#!cp vision/references/detection/transforms.py ./
#!cp vision/references/detection/coco_eval.py ./
#!cp vision/references/detection/engine.py ./
#!cp vision/references/detection/coco_utils.py ./

#### Imports

In [6]:
# Import Torch Libraries
import torch
import torchvision
from torchvision import models, transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# Import Useful Libraries
import os
from engine import train_one_epoch, evaluate, calculate_loss
import utils
import transforms as T

# Import Image/File Work
from PIL import Image
import json
import re
from save_print_magic import tee
import IPython
import numpy as np
import albumentations as A

# Tensorboard
from torch.utils.tensorboard import SummaryWriter

In [7]:
IPython.core.magic.register_cell_magic(tee)

<function save_print_magic.tee(line, cell)>

#### Defining Dataset & Classes

In [8]:
classes = ('NonOverfilled', 'Overfilled')
num_classes = 3
ds = "HRI" # ASL or HRI

#### Dataset Preparation

In [9]:
def DataPrep(image_path, annot_path):
   
    jpg_files = [pos_jpg for pos_jpg in os.listdir(image_path) if pos_jpg.endswith('.jpg')]
    
    with open(annot_path) as json_file:
        annotations = json.load(json_file)
    
    return jpg_files, annotations

In [10]:
def Albu(image):
    # mean = [0.38988095, 0.39120581, 0.3702668]
    # std = [0.18708818, 0.18951494, 0.21228993]
    mean = [0.46917919, 0.47479361, 0.46457069]
    std = [0.28852034, 0.26858532, 0.26505203]

    image = np.array(image)
    #transform = A.CLAHE(clip_limit=(1,4), tile_grid_size=(6,6), always_apply=True, p=1.0)
    transform = A.Normalize(mean=mean, std=std, max_pixel_value=255, always_apply=True, p=1.0)
    aug = transform(image=image)['image']
    #transform2 = A.UnsharpMask(blur_limit=(3,7), sigma_limit=0, alpha=(0.2,0.5), threshold=10, always_apply=True, p=1.0)
    #aug = transform2(image=image)['image']
    #image = Image.fromarray(aug)

    return aug

In [11]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, imSize, imDir, imNames, annots, transforms=None):
        self.height = imSize[0]
        self.width = imSize[1]
        self.imDir = imDir
        self.imNames = imNames
        self.annots = annots
        self.transform = transforms
    
    def __getitem__(self, index):
        image_name = self.imNames[index]
        image_path = f"{self.imDir}{image_name}"

        image = Image.open(image_path).convert('RGB')
        #w, h = image.size
        image = image.resize((self.width, self.height), resample=Image.Resampling.BILINEAR)
        image = Albu(image)

        for item in self.annots['images']:
            if item['file_name'] == image_name:
                imageID = item['id']
                break

        boxes = []
        area = []
        iscrowd = []
        labels = []
        
        for annotation in self.annots['annotations']:
            if annotation['image_id'] == imageID:
                xmin = annotation['bbox'][0]# * self.width/w
                ymin = annotation['bbox'][1] #* self.height/h
                xmax = xmin + annotation['bbox'][2]# * self.width/w
                ymax = ymin + annotation['bbox'][3] #* self.height/h
                # xmin = 0
                # ymin = 0
                # xmax = self.width
                # ymax = self.height
                
                # Append Values to Arrays
                area.append(annotation['area'])
                iscrowd.append(annotation['iscrowd'])
                labels.append(annotation['category_id'])
                boxes.append((xmin, ymin, xmax, ymax))

        # Transform data into Tensor Format
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        imageID = torch.as_tensor(imageID, dtype=torch.int64) # torch.tensor([imageID])
        area = torch.as_tensor(area, dtype=torch.float32)
        iscrowd = torch.as_tensor(iscrowd, dtype=torch.int64)
        
        targets = {}
        targets['boxes'] = boxes
        targets['labels'] = labels
        targets['image_id'] = imageID
        targets['area'] = area
        targets['iscrowd'] = iscrowd

        if self.transform is not None:
            image, targets = self.transform(image, targets)

        return image, targets

    def __len__(self):
        return len(self.imNames)


In [12]:
def get_transform():
    transforms = []
    transforms.append(T.ToTensor())
    return T.Compose(transforms)

In [13]:
# input_image_path = './Dataset/HRI/Control/Train/'

# for filename in os.listdir(input_image_path):
#     os.rename(os.path.join(input_image_path, filename), os.path.join(input_image_path, filename[:-3] + "jpg"))

#### Training Data Loader

In [14]:
image_path = f'./Dataset/{ds}/Reduced/Train/'
annot_path = f'./Dataset/{ds}/Reduced/split_train.json'
imSize = [480, 704]
#imSize = [4000, 6016]

# image_path = f'./Dataset/{ds}/Reduced CROP/Train/'
# annot_path = f'./Dataset/{ds}/Reduced CROP/split_train_crop.json'
# imSize = [236,166]

# image_path = f'./Dataset/{ds}/CROP/Train/'
# annot_path = f'./Dataset/{ds}/CROP/split_train_crop.json'
# imSize = [223, 183]

train_imNames, train_annotations = DataPrep(image_path, annot_path)
train_data = Dataset(imSize, image_path, train_imNames, train_annotations, get_transform())
print('Length of dataset: ', len(train_data), '\n')

image, annotations = train_data[6]
print('Image shape: ', image.shape)                                                                   
print('Annotation Example: ', annotations)
print('Image type: ',image.dtype)

batchSize = 2
workers = 1
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batchSize, shuffle=True, pin_memory=True, num_workers=workers, collate_fn=utils.collate_fn)

Length of dataset:  205 

Image shape:  torch.Size([3, 480, 704])
Annotation Example:  {'boxes': tensor([[408.1702, 153.9600, 570.2447, 438.8400]]), 'labels': tensor([2]), 'image_id': tensor(187), 'area': tensor([3287990.]), 'iscrowd': tensor([0])}
Image type:  torch.float32


#### Validation Data Loader

In [15]:
image_path = f'./Dataset/{ds}/Reduced/Val/'
annot_path = f'./Dataset/{ds}/Reduced/split_val.json'
imSize = [480, 704]
#imSize = [4000, 6016]

# image_path = f'./Dataset/{ds}/Reduced CROP/Val/'
# annot_path = f'./Dataset/{ds}/Reduced CROP/split_val_crop.json'
# imSize = [236, 166]

# image_path = f'./Dataset/{ds}/SHARP_EDGE/Val/'
# annot_path = f'./Dataset/{ds}/CROP/split_val_crop.json'
# imSize = [223, 183]

val_imNames, val_annotations = DataPrep(image_path, annot_path)

val_data = Dataset(imSize, image_path, val_imNames, val_annotations, get_transform())

batchSize = 1
workers = 1
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batchSize, shuffle=False, pin_memory=True, num_workers=workers, collate_fn=utils.collate_fn)

#### Test Data Loader

In [16]:
# image_path = f'./Dataset/{ds}/Test/'
# annot_path = f'./Dataset/{ds}/split_test.json'
# imSize = [480, 704]

# # image_path = f'./Dataset/{ds}/SHARP_EDGE/Test/'
# # annot_path = f'./Dataset/{ds}/CROP/split_test_crop.json'
# # imSize = [223, 183]

# test_imNames, test_annotations = DataPrep(image_path, annot_path)

# test_data = Dataset(imSize, image_path, test_imNames, test_annotations, get_transform())

# batchSize = 1
# workers = 1
# test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=batchSize, shuffle=False, pin_memory=True, num_workers=workers, collate_fn=utils.collate_fn)

***
## Training

#### Define Model

In [17]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print(device)

cuda


In [18]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights=models.detection.FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.to(device)

learning_rate = 0.01

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=learning_rate, momentum=0.9, weight_decay=0.0001) # momentum=0.9,

lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[6,10], gamma=0.1)

#### Tensorboard For Plotting Training/Validation Loss & Accuracy

In [19]:
writer = SummaryWriter()

#### Training Function

In [20]:
%%tee cap
#%%capture cap --no-stderr

epochs = 12
dl_len = len(train_data_loader)
step = 0

for epoch in range(epochs):
    train_one_epoch(step, writer, model, optimizer, train_data_loader, device, epoch, print_freq=40)
    lr_scheduler.step()
    # print('\n\n-------------------- Training Evaluation --------------------')
    # evaluate(model, train_data_loader, device=device)
    print('\n\n------------------ Validation Evaluation -------------------')
    evaluate(model, val_data_loader, device=device)
    # print('\n\n---------------------- Test Evaluation ----------------------')
    # evaluate(model, test_data_loader, device=device)

    validation_loss = calculate_loss(model, val_data_loader, device=device)
    step = ((epoch + 1) * dl_len)
    writer.add_scalars('Training vs Validation Loss', {'Validation': validation_loss}, step)

    print('\n')

torch.save(model, f'./Outputs/{ds}_Epoch{epochs}_{learning_rate}.pt')

print('Training is now finished')

Epoch: [0]  [  0/103]  eta: 0:02:20  lr: 0.000108  loss: 1.4307 (1.4307)  loss_classifier: 1.1824 (1.1824)  loss_box_reg: 0.1599 (0.1599)  loss_objectness: 0.0839 (0.0839)  loss_rpn_box_reg: 0.0046 (0.0046)  time: 1.3633  data: 0.0778  max mem: 3612
Epoch: [0]  [ 40/103]  eta: 0:00:36  lr: 0.004026  loss: 0.3564 (0.5594)  loss_classifier: 0.1261 (0.3383)  loss_box_reg: 0.2199 (0.1947)  loss_objectness: 0.0115 (0.0232)  loss_rpn_box_reg: 0.0015 (0.0032)  time: 0.5671  data: 0.0001  max mem: 3874
Epoch: [0]  [ 80/103]  eta: 0:00:13  lr: 0.007943  loss: 0.1506 (0.3829)  loss_classifier: 0.0738 (0.2119)  loss_box_reg: 0.0689 (0.1527)  loss_objectness: 0.0053 (0.0158)  loss_rpn_box_reg: 0.0015 (0.0026)  time: 0.5676  data: 0.0001  max mem: 3874
Epoch: [0]  [102/103]  eta: 0:00:00  lr: 0.010000  loss: 0.1093 (0.3278)  loss_classifier: 0.0564 (0.1787)  loss_box_reg: 0.0459 (0.1334)  loss_objectness: 0.0023 (0.0132)  loss_rpn_box_reg: 0.0014 (0.0024)  time: 0.5534  data: 0.0001  max mem: 3874


#### Save Training Function Cell Output to TXT File

In [21]:
with open('./Outputs/output.txt', 'w') as f:
    f.write(str(cap))

#### Extract mAP from Output File

In [22]:
mAP = re.compile(r'^ Average Precision.*IoU=0.50:0.95.*=   all')
mAP_05 = re.compile(r'^ Average Precision.*IoU=0.50 .*=   all')
mAP_075 = re.compile(r'^ Average Precision.*IoU=0.75.*=   all')
mAP_s = re.compile(r'^ Average Precision.*IoU=0.50:0.95.*= small')
mAP_m = re.compile(r'^ Average Precision.*IoU=0.50:0.95.*=medium')
mAP_l = re.compile(r'^ Average Precision.*IoU=0.50:0.95.*= large')

train_mAP_list = []
train_mAP_05_list = []
train_mAP_075_list = []
train_mAP_s_list = []
train_mAP_m_list = []
train_mAP_l_list = []

val_mAP_list = []
val_mAP_05_list = []
val_mAP_075_list = []
val_mAP_s_list = []
val_mAP_m_list = []
val_mAP_l_list = []

test_mAP_list = []
test_mAP_05_list = []
test_mAP_075_list = []
test_mAP_s_list = []
test_mAP_m_list = []
test_mAP_l_list = []

step = 0
flag = 2

with open('./Outputs/output.txt') as f:
    for line in f:
        if mAP.match(line):
            step = step + len(train_data_loader)
            match flag:
                #case 1:
                #    train_mAP_list.append(float(line[74:79]))
                #    writer.add_scalars('Training vs Validation mAP', {'train_mAP': float(line[74:79])}, step)

                case 2:
                    val_mAP_list.append(float(line[74:79]))
                #    writer.add_scalars('Training vs Validation mAP', {'val_mAP': float(line[74:79])}, step)
                    writer.add_scalars('Validation vs Test mAP', {'val_mAP': float(line[74:79])}, step)

                case 3:
                    test_mAP_list.append(float(line[74:79]))
                    writer.add_scalars('Validation vs Test mAP', {'test_mAP': float(line[74:79])}, step)

            
        if mAP_05.match(line):
            match flag:
                #case 1:
                #    train_mAP_05_list.append(float(line[74:79]))

                case 2:
                    val_mAP_05_list.append(float(line[74:79]))

                case 3:
                    test_mAP_05_list.append(float(line[74:79]))

        if mAP_075.match(line):
            match flag:
                #case 1:
                #    train_mAP_075_list.append(float(line[74:79]))

                case 2:
                    val_mAP_075_list.append(float(line[74:79]))

                case 3:
                    test_mAP_075_list.append(float(line[74:79]))

        if mAP_s.match(line):
            match flag:
                #case 1:
                #    train_mAP_s_list.append(float(line[74:79]))
 
                case 2:
                    val_mAP_s_list.append(float(line[74:79]))

                case 3:
                    test_mAP_s_list.append(float(line[74:79]))

        if mAP_m.match(line):
            match flag:
                #case 1:
                #    train_mAP_m_list.append(float(line[74:79]))

                case 2:
                    val_mAP_m_list.append(float(line[74:79]))

                case 3:
                    test_mAP_m_list.append(float(line[74:79]))

        if mAP_l.match(line):
            match flag:
                #case 1:
                #    train_mAP_l_list.append(float(line[74:79]))
                #    flag = 2
                case 2:
                    val_mAP_l_list.append(float(line[74:79]))
                    flag = 3
                case 3:
                    test_mAP_l_list.append(float(line[74:79]))
                    flag = 2

In [23]:
writer.close()

In [24]:
with open('./Outputs/mAP_file.txt', 'w') as f:
    f.write('---------------------------- Training mAPs ----------------------------\n')
    f.write(f'mAP = {train_mAP_list}\n')
    f.write(f'mAP_05 = {train_mAP_05_list}\n')
    f.write(f'mAP_075 = {train_mAP_075_list}\n')
    f.write(f'mAP_s = {train_mAP_s_list}\n')
    f.write(f'mAP_m = {train_mAP_m_list}\n')
    f.write(f'mAP_l = {train_mAP_l_list}\n\n\n')
    f.write('--------------------------- Validation mAPs ---------------------------\n')
    f.write(f'mAP = {val_mAP_list}\n')
    f.write(f'mAP_05 = {val_mAP_05_list}\n')
    f.write(f'mAP_075 = {val_mAP_075_list}\n')
    f.write(f'mAP_s = {val_mAP_s_list}\n')
    f.write(f'mAP_m = {val_mAP_m_list}\n')
    f.write(f'mAP_l = {val_mAP_l_list}\n\n\n')
    f.write('------------------------------ Test mAPs ------------------------------\n')
    f.write(f'mAP = {test_mAP_list}\n')
    f.write(f'mAP_05 = {test_mAP_05_list}\n')
    f.write(f'mAP_075 = {test_mAP_075_list}\n')
    f.write(f'mAP_s = {test_mAP_s_list}\n')
    f.write(f'mAP_m = {test_mAP_m_list}\n')
    f.write(f'mAP_l = {test_mAP_l_list}')

    f.close()

***
## Hyperparameter Finding Functions

In [25]:
#from torch_lr_finder import LRFinder
#import torch.nn as nn

#criterion = nn.CrossEntropyLoss()

#lr_finder = LRFinder(model, optimizer, criterion, device="cuda")
#lr_finder.range_test(train_data_loader, val_loader=val_data_loader, end_lr=1, num_iter=100, step_mode="linear")
#lr_finder.plot(log_lr=False)
#lr_finder.reset()

In [26]:
#from time import time
#import multiprocessing as mp

#for num_workers in range(2, mp.cpu_count(), 2):  
#    train_loader = DataLoader(train_reader,shuffle=True,num_workers=num_workers,batch_size=64,pin_memory=True)
#    start = time()
#    for epoch in range(1, 3):
#        for i, data in enumerate(train_loader, 0):
#            pass
#    end = time()
#    print("Finish with:{} second, num_workers={}".format(end - start, num_workers))